In [1]:
import pandas as pd
import pickle
import csv
import os
import re
import emoji
from textblob import TextBlob
import nltk
from langdetect import detect
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix
import gensim
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
## Cleaning methods
def clean_emoji(text):
    text=emoji.demojize(text)
    return text
def remove_url(text):
    text = ' '.join(re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"," ", text).split())
    return text
def remove_symbol(text):
    text = str.replace(text,',',' ')
    text = str.replace(text,'+',' ')
    text = str.replace(text,'=',' ')
    text = ' '.join(re.sub("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"," ", text).split())
    text = str.replace(text,"'",' ')
    text = str.replace(text,'"',' ')
    text = str.replace(text,'!',' ')
    text = str.replace(text,'^',' ')
    text = str.replace(text,'(',' ')
    text = str.replace(text,')',' ')
    text = str.replace(text,'%',' ')
    text = str.replace(text,'-',' ')
    text = str.replace(text,'_',' ')
    text = str.replace(text,'|',' ')
    text = str.replace(text,'.',' ')
    text = str.replace(text,':',' ')
    return text    
## this is the optional function we can use to remove @ and other. Might not use it if we need to analyze retweets
def optional_rm(text):
    all_words = text.split(' ')
    remove_words = []
    for each in all_words:
        if each == ' ' or each == '':
            continue
        if each[0] == '#' or each[0] == '@':
            remove_words.append(each)
            if each[0] == '#':
                hashtags = each.split('#')
                for tag in hashtags:
                    gl_hashtag.append(tag.lower())

    for word in remove_words:
        all_words.remove(word)
    text = ' '.join(all_words)
    
    text = text.split(' ')
    new_text = []
    for each in text:
        if(str.find(each,'http') != -1):
            continue
        
        if not each.isalnum():
            continue
        new_text.append(str.lower(each));
    text = ' '.join(new_text)
    
    return text


def is_english(text):
    try:
        if detect(text) == "en":
            return True
        else:
            return False
    except Exception as e:
        return True
        print(e)   
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True    
def processing(text):
    #text=clean_emoji(text)
    #text=remove_url(text)
    #text = remove_symbol(text)
    text = optional_rm(text)
    return text

In [3]:
## Import Data
def read_json_to_dataframe(filename):
    print(filename)
    #f = open(filename,encoding = 'utf-16').read()
    f1 = pd.read_json(filename, orient='records',lines=True,encoding = 'utf-16')
    return f1

In [162]:
df = pd.read_json(r'C:\Users\luoyu\Desktop\USCISI\vaccine hesitancy\raw\10-19-2020.json', orient = 'records',lines=True, encoding = 'utf-16')

In [4]:
def read_in(path):
    files = os.listdir(path)
    all_dfs = []
    print(files)
    for i in files:
        if 'json' in i:
            each_df = read_json_to_dataframe(path+i)
            all_dfs.append(each_df)
            
    return pd.concat(all_dfs, ignore_index=True)

all_df = read_in('C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/')
all_df.shape

['10-19-2020.json', '10-20-2020.json', '10-21-2020.json', '10-22-2020.json', '10-23-2020.json', '10-24-2020.json', '10-25-2020.json']
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-19-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-20-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-21-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-22-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-23-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-24-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-25-2020.json


(371940, 23)

In [5]:
## Get rid of the folded tweets
test_df = all_df.copy()
test_df=test_df[~test_df['og_tweet_txt'].str.endswith('\u2026',na=True)]
test_df=test_df[~((test_df['tweet_txt'].str.endswith('\u2026',na=True)) & (test_df['og_tweet_txt'].str.strip() == ""))]
# test_df=test_df.dropna(axis=0,how='any')

In [137]:
with open("./cleaned.pkl", 'rb') as f:
    clean_df = pickle.load(f)

In [138]:
test_df = clean_df.copy()

In [69]:
no_retweet = test_df[test_df['og_tweet_time'].isna()].copy()
retweet = test_df[test_df['og_tweet_time'].notna()].copy()

In [173]:
lstm_pred = pd.read_csv(r"C:\Users\luoyu\Desktop\USCISI\vaccine hesitancy\lstm_pred\LSTM_prediction.csv")

In [247]:
cluster_df_sample_6x100 = pd.read_excel(r"C:\Users\luoyu\Desktop\USCISI\vaccine hesitancy\cluster_df_sample_6x100.xlsx")
cluster_df_sample_6x100 = cluster_df_sample_6x100[['combined_tweet_txt','true_class']]
top_ambiguous = pd.read_excel(r"C:\Users\luoyu\Desktop\USCISI\vaccine hesitancy\top_ambiguous.xlsx")
for_join = cluster_df_sample_6x100.append(top_ambiguous[['combined_tweet_txt','true_class']]).copy()
for_join = for_join[for_join.true_class != 0]
joined_df = for_join.set_index('combined_tweet_txt').join(lstm_pred.set_index('combined_tweet_txt'), how='inner')[['true_class','tweet_id']]
joined_df = joined_df[~joined_df.index.duplicated(keep='first')]
joined_df_2 = joined_df.set_index('tweet_id').join(all_df.set_index('tweet_id'), how='left')

In [288]:
global gl_hashtag
stop_hash_word = ['COVID19']
for i in range(6):
    stop_hash_word = ['','covid19','covid-19','vaccine','coronavirus','covid','covid-19','china\n\nhttps://t.co/mg6dph4xp7','edappadipalaniswami', 'bo…']
    selected_class = joined_df_2[joined_df_2.true_class == i+1]
    gl_hashtag = []
    clean_df = clean_txt(selected_class)
    for word in stop_hash_word:
        gl_hashtag = list(filter(lambda a: a != word, gl_hashtag))
    print(len(df_tags['hashtag'].unique()))
    print(len(df_tags))
    print(len(selected_class))
    df_tags = pd.DataFrame(gl_hashtag,columns=['hashtag'])
    df_count = df_tags.groupby(['hashtag']).size().reset_index(name='counts')
    print(list(df_count.sort_values(by='counts',ascending = False).head(20)['hashtag']))

2
2
45
['debates2020:', 'freedom', 'trump', 'trumpliesamericansdie', 'votebluedownballot', 'votehimout2020']
6
6
77
['british', 'coronavirus.”', 'newsrail:']
3
3
174
['astrazeneca', 'pharma', 'uk', 'th…', 'stopthemadness\n', 'socialmedia', 'saynotothevaccine\nvolunteer', 'riodejaneiro', 'pmlive', 'new:', 'bmj', 'healthmrx', 'health', 'globalbuzz', 'cyberattacks…', 'covidiots', 'coronavirus:', 'brazil', 'wakeupeverybody\n']
19
21
84
['biharelections', 'biharelections2020', 'biofarmaterpercaya', 'bjp', 'bjp…', 'covid_19', 'covidー19', 'gatesfoundation', 'jio', 'leronlimab', 'manifesto', 'reliance', 'trumpvirus!', 'uk', 'worstpresidentinhistory…']
15
15
14
['meatballarmy']
1
1
18
['flushot', 'kayburley']


<ipython-input-199-dc682236291b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_txt'] = df['tweet_txt'].apply(lambda x: processing(x))
<ipython-input-199-dc682236291b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_txt'] = df['tweet_txt'].apply(lambda x: processing(x))
<ipython-input-199-dc682236291b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [359]:
stopword=['going','like','think','ever','still','make','one','could','even','take','people','good','says','developed','get','may',"i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
stop_hash_word = ['vaccines','year','rt','covid19','virus','via','covid-19','vaccine','coronavirus','covid','covid-19','china\n\nhttps://t.co/mg6dph4xp7','edappadipalaniswami', 'bo…','19', 'bo', 'china', 'co', 'https', 'mg6dph4xp7']
stopword.extend(stop_hash_word)

In [360]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm='l2',use_idf=True,max_features=500,stop_words=stopword,ngram_range=(1,2))
tfIdf = vectorizer.fit_transform(joined_df_2['tweet_txt'])

In [361]:
vocabs = vectorizer.get_feature_names()
row_keys=[]
for i in range(412):
    row_keys.append([vocabs[i] for i in (tfIdf[i]).nonzero()[1]])
joined_df_2['keywords'] = row_keys

In [362]:
class_keywords = [item for sublist in list(joined_df_2['keywords']) for item in sublist]
top_keywords = [key for key, value in Counter(class_keywords).most_common()][:50]

In [363]:
from collections import Counter
keyword_each_class=[]
for i in range(6):
    selected_class = joined_df_2[joined_df_2.true_class == i+1]
    class_keywords = [item for sublist in list(selected_class['keywords']) for item in sublist]
    top_keywords = [key for key, value in Counter(class_keywords).most_common()][:20]
    keyword_each_class.append(top_keywords)

In [364]:
keyword_each_class

[['trump',
  'office',
  'safety',
  'safety office',
  'last',
  'shut',
  'office last',
  'shut safety',
  'administration',
  'trump administration',
  'administration shut',
  'plan',
  'closed',
  'last plan',
  'cuomo',
  'ready',
  'skeptical',
  'also',
  'gates',
  'long'],
 ['never',
  'flu',
  'hiv',
  'disease',
  'hopes',
  'pinning',
  're',
  'drug',
  'long',
  'around',
  'dose realism',
  'hopes dose',
  'pinning hopes',
  're pinning',
  'dose',
  'realism',
  'amp',
  'rushed',
  'disappear',
  'might'],
 ['trial',
  'volunteer',
  'astrazeneca',
  'brazil',
  'dies',
  'oxford',
  'died',
  'astrazeneca trial',
  'trial dies',
  'dies brazil',
  'death',
  'trials',
  'university',
  'volunteers',
  'volunteer astrazeneca',
  'oxford trial',
  'uk',
  'volunteer oxford',
  'trial volunteer',
  'volunteer dies'],
 ['government',
  'gates',
  'amp',
  'bill',
  'volunteers',
  'bill gates',
  'us',
  'trust',
  'shot',
  'called',
  'bjp',
  'shut',
  'trump',
  'ex

In [199]:
def clean_txt(df):
    #df["og_tweet_txt"] = df['og_tweet_txt'].apply(lambda x: processing(x))
    df['tweet_txt'] = df['tweet_txt'].apply(lambda x: processing(x))
    #df[df["tweet_txt"].apply(is_english)]
    #df[df["og_tweet_txt"].apply(is_english)]
    return df

In [78]:
global gl_hashtag
gl_hashtag = []
clean_df = clean_txt(test_df)
gl_hashtag = list(filter(lambda a: a != '', gl_hashtag))

In [79]:
df_tags = pd.DataFrame(gl_hashtag,columns=['hashtag'])
print(len(df_tags['hashtag'].unique()))
print(len(df_tags))
df_count = df_tags.groupby(['hashtag']).size().reset_index(name='counts')
df_count.sort_values(by='counts',ascending = False).head(20)

26949
158942


,hashtag,counts
2259,COVID19,10625
25958,vaccine,7569
2255,COVID,4187
18312,coronavirus,3702
3457,Covid19,2551
2479,COVIDVaccine,1927
3456,Covid,1864
16216,WhatsHappeningInMyanmar,1814
15386,Vaccine,1694
18400,covid,1629


In [45]:
list(df_count.sort_values(by='counts',ascending = False).head(50)['hashtag'])

['COVID19',
 'vaccine',
 'COVID',
 'coronavirus',
 'Covid19',
 'COVIDVaccine',
 'Covid',
 'WhatsHappeningInMyanmar',
 'Vaccine',
 'covid',
 'Coronavirus',
 'CCPVirus',
 'DrLiMengYan1',
 'TakeDownTheCCP',
 '爆料革命',
 '郭文贵',
 'COVIDー19',
 'YanLiMeng',
 '闫丽梦',
 'covid19',
 'vaccines',
 'Pfizer',
 'COVID19Vaccine',
 'news',
 'auspol',
 'CovidVaccine',
 'Feb20Coup',
 'Health',
 'vaccination',
 'Feb24Coup',
 'BREAKING',
 'cdnpoli',
 'pandemic',
 'NHS',
 'COVAX',
 'PeoplesVaccine',
 'vaccinated',
 'lockdown',
 'SmartNews',
 'India',
 'AstraZeneca',
 'Israel',
 'cvspharmtech',
 'WhatsHappeninglnMyanmar',
 'pharmtech',
 'WTO',
 'healthcare',
 'CrimesAgainstHumanity',
 'Biden',
 'News']

In [3]:
lstm_pred = pd.read_csv(r"C:\Users\luoyu\Desktop\USCISI\vaccine hesitancy\lstm_pred\LSTM_prediction.csv")

In [15]:
len(lstm_pred.loc[lstm_pred['prediction'] == -1])/len(lstm_pred)

0.23909710916038648

In [14]:
len(lstm_pred.loc[lstm_pred['prediction'] == -1])

85471

In [18]:
with open("./cleaned.pkl", 'rb') as f:
    clean_df = pickle.load(f)

In [140]:
len(no_retweet['user_id'].unique())

72879

In [88]:
lstm_pred = pd.read_excel('./cutoff50.xlsx')
lstm_pred = lstm_pred.drop(columns='Unnamed: 0')

In [92]:
lstm_pred = lstm_pred.drop(columns='Unnamed: 0')

In [94]:
no_retweet = lstm_pred[lstm_pred['og_tweet_id'].isna()].copy()
retweet = lstm_pred[lstm_pred['og_tweet_id'].notna()].copy()

In [84]:
lstm_pred = lstm_pred.drop('og_tweet_id',axis=1)
prediction_joined = test_df.set_index('tweet_id').join(lstm_pred.set_index('tweet_id'))

In [85]:
no_retweet = prediction_joined[prediction_joined['og_tweet_time'].isna()].copy()
retweet = prediction_joined[prediction_joined['og_tweet_time'].notna()].copy()

In [101]:
print(len(retweet.loc[retweet['prediction'] == -1])/len(retweet))
print(len(retweet.loc[retweet['prediction'] == -1]))
print(len(no_retweet.loc[no_retweet['prediction'] == -1])/len(no_retweet))
print(len(no_retweet.loc[no_retweet['prediction'] == -1]))

0.2519198940748097
60884
0.19541213493071274
14158


In [104]:
print(len(lstm_pred.loc[lstm_pred['prediction'] == 0])/len(lstm_pred))
print(len(lstm_pred.loc[lstm_pred['prediction'] == 0]))

0.7521583283460456
236277


In [106]:
len(retweet)

241680